# Functions for Courses File

In [3]:
import pandas as pd
import sqlite3
import courses
from importlib import reload

In [ ]:
# Run this if you change the 'courses.py' file but have already created the database in this notebook
reload(courses);

In [4]:
# connect to the database
# use create = True only the very first time running
db = courses.CoursesDB('courses.db', create = False) 

In [5]:
# run this box on your first time connecting to the database
# OR if you change something in the 'courses.py' file related to building the tables
db.drop_all_tables(are_you_sure = True)
db.build_tables()

In [6]:
# load the data for any meets you want to look at
# use db.load_results('link')
db.load_results('https://www.tfrrs.org/results/xc/23362/2024_Pirate_Cross_Country_Invitational_')
db.load_results('https://www.tfrrs.org/results/xc/25186/Panorama_Farms_Invitational')

In [9]:
# all the data together will look something like this
db.run_query('''SELECT * FROM tRunner
JOIN tRaceResult USING(runner_id)
JOIN tRace USING(race_id)''')

,runner_id,name,eligibility,school,race_id,raw_time,time,place,race,date
0,1,Peninah Mutisya,FR-1,Hampton,1,20:54.4,1254.4,1,2024 Pirate Cross Country Invitational,"October 4, 2024"
1,2,Kyra Holland,SR-4,William & Mary,1,21:07.1,1267.1,2,2024 Pirate Cross Country Invitational,"October 4, 2024"
2,3,Sofia Istnick,JR-3,William & Mary,1,21:35.8,1295.8,3,2024 Pirate Cross Country Invitational,"October 4, 2024"
3,4,Arianna DeBoer,SR-4,William & Mary,1,21:39.3,1299.3,4,2024 Pirate Cross Country Invitational,"October 4, 2024"
4,5,Sarah McCartney,SR-4,Charleston Southern,1,21:42.4,1302.4,5,2024 Pirate Cross Country Invitational,"October 4, 2024"
...,...,...,...,...,...,...,...,...,...,...
338,279,Nathalie Lavender,JR-3,VMI,2,26:23.6,1583.6,130,Panorama Farms Invitational,"October 19, 2024"
339,280,Natalia Dias,FR-1,VMI,2,26:52.9,1612.9,131,Panorama Farms Invitational,"October 19, 2024"
340,281,Samantha Layton,FR-1,VMI,2,27:00.1,1620.1,132,Panorama Farms Invitational,"October 19, 2024"
341,282,Caroline Fiorillo,SR-4,VMI,2,27:18.7,1638.7,133,Panorama Farms Invitational,"October 19, 2024"


In [7]:
# see which races you have loaded
db.see_loaded_races()

,race_id,race,date
0,1,2024 Pirate Cross Country Invitational,"October 4, 2024"
1,2,Panorama Farms Invitational,"October 19, 2024"


In [10]:
# This function compares two courses specified by their race_id's.
# It will output the difference in seconds in average race times (difference), the ratio of average race times (ratio), and
# the number of runners in common between the two courses (NumCompared).
# The first course is used as a comparison point. 'difference' is the number of seconds faster or slower that the second course
# averages compared to the first course; a negative value for 'difference' means the second course was faster.
# 'ratio' is the number that times from the second course would need to be multiplied by in order to standardize them to the first
# course; the average time from the second course multiplied by 'ratio' should yield the average time from the first course.
# This function only compares times in runners who competed in both meets. The number of runners in common is shown as NumCompared.
db.compare_two_courses(1,2)

,Difference,Ratio
0,-31.133333,0.977666
